<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/T5_F3_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de bibliotecas necesarias
!pip install --upgrade transformers
!pip install transformers pandas torch scikit-learn bert-score optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
# Importación de bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
import math
import numpy as np
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu
import optuna

In [ ]:
# Configuración de hiperparámetros iniciales
HYPERPARAMS = {
    "model_name": "t5-small",
    "max_length": 128,
    "batch_size": 2,
    "num_epochs": 10,
    "learning_rate": 5e-5,
    "num_beams": 5,
    "no_repeat_ngram_size": 2,
    "early_stopping": True,
    "logging_steps": 100,
    "output_dir": "./noticias_t5_model",
    "logging_dir": "./logs",
    "save_steps": 500,
    "save_total_limit": 2
}

In [ ]:
# Cargar datos
from google.colab import files
uploaded = files.upload()

data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [ ]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row['Año']}, Mes: {row['Mes']}, "
        f"Trimestre: {row['Trimestre']}, Frecuencia: {row['Frecuencia']}, "
        f"Año_comparación: {row['Año_comparación']}, País: {row['País']}, "
        f"Tendencia: {row['Tendencia']}, Sector: {row['Sector']}, "
        f"Indicador: {row['Indicador']}, Valor_actual: {row['Valor_actual']}, "
        f"Variación: {row['Variación']}, Valor_comparación: {row['Valor_comparación']}, "
        f"Entidad: {row['Entidad']}"
    )
    return texto_datos

data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]

In [ ]:
# Clase Dataset personalizada
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entrada_texto = self.data.iloc[idx]["entrada_texto"]
        noticia = self.data.iloc[idx]["Noticia"]

        input_encoding = self.tokenizer(
            entrada_texto,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            noticia,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": target_encoding["input_ids"].flatten(),
        }

In [ ]:
# Dividir datos en entrenamiento (80%) y evaluación (20%)
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)
eval_data = eval_data.reset_index(drop=True)

In [ ]:
# Cargar tokenizador y modelo preentrenado
tokenizer = T5Tokenizer.from_pretrained(HYPERPARAMS["model_name"])
model = T5ForConditionalGeneration.from_pretrained(HYPERPARAMS["model_name"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Crear datasets
train_dataset = NoticiasDataset(train_data, tokenizer, HYPERPARAMS["max_length"])
eval_dataset = NoticiasDataset(eval_data, tokenizer, HYPERPARAMS["max_length"])
eval_dataloader = DataLoader(eval_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=False)

In [ ]:
# Funciones para evaluación
def calcular_perplejidad(model, tokenizer, eval_dataloader):
    model.eval()
    total_loss = 0
    total_tokens = 0
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss if hasattr(outputs, 'loss') else outputs[0]
            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)
    return math.exp(total_loss / total_tokens)

def calcular_bleu_score(model, tokenizer, eval_dataloader):
    model.eval()
    bleu_scores = []
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)
        with torch.no_grad():
            outputs = model.generate(input_ids, attention_mask=attention_mask)
            decoded_preds = tokenizer.decode(outputs[0], skip_special_tokens=True)
            decoded_labels = tokenizer.decode(labels[0], skip_special_tokens=True)
            bleu_scores.append(sentence_bleu([decoded_labels.split()], decoded_preds.split()))
    return np.mean(bleu_scores)

def calcular_bertscore(model, tokenizer, dataset):
    model.eval()
    referencias = []
    generadas = []
    for i in range(len(dataset)):
        datos = eval_data.iloc[i]
        referencias.append(datos["Noticia"])
        generadas.append(generar_noticia(model, tokenizer, datos))
    P, R, F1 = bert_score(generadas, referencias, lang="es", model_type="bert-base-uncased")
    return {
        "Precision promedio": P.mean().item(),
        "Recall promedio": R.mean().item(),
        "F1 promedio": F1.mean().item(),
    }

In [ ]:
# Función para generación de noticias
def generar_noticia(model, tokenizer, datos):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer.encode(entrada_texto, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs,
        max_length=HYPERPARAMS["max_length"],
        num_beams=HYPERPARAMS["num_beams"],
        no_repeat_ngram_size=HYPERPARAMS["no_repeat_ngram_size"],
        early_stopping=HYPERPARAMS["early_stopping"],
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Optimización Bayesiana con Optuna
def objective(trial):
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
        "batch_size": trial.suggest_categorical("batch_size", [2, 4, 8]),
        "num_epochs": trial.suggest_int("num_epochs", 5, 15),
        "num_beams": trial.suggest_int("num_beams", 3, 7),
        "no_repeat_ngram_size": trial.suggest_int("no_repeat_ngram_size", 2, 3),
    }

    HYPERPARAMS.update(params)

    eval_dataloader = DataLoader(eval_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=False)

    training_args = TrainingArguments(
        output_dir=HYPERPARAMS["output_dir"],
        num_train_epochs=params["num_epochs"],
        per_device_train_batch_size=params["batch_size"],
        save_steps=HYPERPARAMS["save_steps"],
        save_total_limit=HYPERPARAMS["save_total_limit"],
        logging_dir=HYPERPARAMS["logging_dir"],
        logging_steps=HYPERPARAMS["logging_steps"],
        learning_rate=params["learning_rate"],
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    trainer.train()
    bleu_score = calcular_bleu_score(model, tokenizer, eval_dataloader)
    return bleu_score

In [ ]:
# Crear estudio de Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Resultados finales
print("Mejores hiperparámetros:")
print(study.best_params)
print(f"Mejor BLEU obtenido: {study.best_value}")

[I 2025-01-03 22:14:27,520] A new study created in memory with name: no-name-df781ddf-6c2b-43b1-a2f5-022c94e0ef09
<ipython-input-12-6e61eee9f4d6>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Step,Training Loss
100,0.026000
200,0.029900
300,0.026000
400,0.026800
500,0.033000
600,0.030900
700,0.028200
800,0.027300
900,0.025400
1000,0.027300


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Step,Training Loss
100,0.024500
200,0.035600
300,0.039200
400,0.035100
500,0.040000
600,0.032500
700,0.041000
800,0.036600
900,0.047400
1000,0.040900


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Mejores hiperparámetros:
{'learning_rate': 5.286122063435074e-05, 'batch_size': 8, 'num_epochs': 14, 'num_beams': 4, 'no_repeat_ngram_size': 3}
Mejor BLEU obtenido: 0.24150851665278633
